In [1]:
import sys
import os

import psycopg2

import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection

from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.prompts import PromptTemplate

import pandas as pd

sys.path.append('../../')
from utils import get_config

sys.path.append('../')
from gis_resources import san_diego_county_zips

In [2]:
nourish_user = get_config("nourish_db","username")
nourish_pswd = get_config("nourish_db","passkey")
ai_key = get_config("open_ai","key")

In [3]:
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [4]:
# 32: Manuel,Gonzalez -> prospective food truck owner
registrant_id = 32

In [5]:
# Get resgistrant attributes
col_qry = """
SELECT
    column_name
FROM
    information_schema.columns
WHERE
    table_name = 'registrants';
"""


cur = conn.cursor()

# execute a statement
cur.execute(col_qry)


# display the PostgreSQL database server version
col_names = cur.fetchall()
col_names = [c[0] for c in col_names]
       
# Close the communication with the PostgreSQL
cur.close()

In [6]:
# Get registrant of interest
# create a cursor
cur = conn.cursor()

query_str = f"select * "\
            f"from registrants " \
            f"where id = {registrant_id}"

# execute a statement
cur.execute(query_str)


# display the PostgreSQL database server version
reg_obs = cur.fetchall()
       
# Close the communication with the PostgreSQL
cur.close()

In [7]:
# make df
df = pd.DataFrame(reg_obs,
              columns=col_names)
df

,id,first_name,last_name,middle_name,email,phone,gender,ethnicity,languages_spoken,languages_written,...,insert_dts,update_dts,home_city,home_state,home_zip,biz_city,biz_state,biz_zip,home_street_address,biz_stree_address
0,32,Manuel,Gonzalez,None,[m.gonzalez@gmail.com],[619-231-4824],Male,[Hispanic],"[Spanish, English]","[Spanish, English]",...,2023-04-16 19:11:50.029601,None,San Diego,CA,92154,[],CA,[],1372-74 Del Sol Ln,None


In [8]:
def convert_df(df):
    assert df.shape[0] == 1
    res_df = df.copy()
    res_df.dropna(inplace=True, axis=1)
    res_df.drop(columns=["id", "insert_dts"], inplace=True)
    return {c.replace("biz", "business"): res_df[c][0] for c in res_df.columns}

In [9]:
# Put together prospective user information to give base context to user
user_info = convert_df(df)


# Initial prompt
prompt = f"Here is some information about a person. There information is given in json format below." \
        "Please welcome this person as if you are talking to them and summarize their business needs, location, and funding needs. " \
        "Please do not repeat their personal information back to them since they already know this and you are talking to them." \
        f"{user_info}"
# prompt = f"This is {df['first_name']} {df['last_name']}. " \
#           "The following information relates to their experience and business goals: " \
#          f"gender: {df['gender']}, ethnicity: {df['ethnicity']}, is vetran: {df['is_veteran']} " \
#          f"business description: {df['current_business_description']} {df['prospective_business_description']}. " \
#          f"Please welcome this person and summarize their business, location, and funding needs for {df['biz_state']}."


# df['prospective_business_description'].values[0]
prompt

"Here is some information about a person. There information is given in json format below.Please welcome this person as if you are talking to them and summarize their business needs, location, and funding needs. Please do not repeat their personal information back to them since they already know this and you are talking to them.{'first_name': 'Manuel', 'last_name': 'Gonzalez', 'email': ['m.gonzalez@gmail.com'], 'phone': ['619-231-4824'], 'gender': 'Male', 'ethnicity': ['Hispanic'], 'languages_spoken': ['Spanish', 'English'], 'languages_written': ['Spanish', 'English'], 'extending_existing_business': False, 'prospective_business_description': 'I would like to set up a small Mexican grocery somewhere within 10 miles from home. There is an old store that is up for sale, and it would be great to buy it. I need to do some renovations to get it ready for business. While my wife and I will work in the store, I need to have a couple of employees to get me supplies. ', 'desired_funding': Decima

In [10]:
llm = OpenAI(temperature=0, verbose=True, openai_api_key=ai_key)

# If you want to test -> uses up $0.00046
# text = "What would be a good company name for a company that makes colorful socks?"
# print(llm(text))

In [11]:
print(llm(prompt))



Welcome Manuel! It sounds like you are looking to set up a small Mexican grocery in San Diego, CA. You are looking to purchase an existing store and need to do some renovations. You also need to hire a couple of employees. You are looking for $70,000 in funding to help you get started.


In [ ]:
# Eventually plan out the tools
# tools = ["Location", "Cuisine"]

In [ ]:
raise NotImplementedError

In [3]:
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=get_config("arcgis","clientid"))

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=nxo9a9QQpwjWn6Fi&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=rb295sl6RN7cDZOqt1cCJAEipcyxQw&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/mpstickle/school/capstone/capstone/arc_gis/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
